In [ ]:
configs = {"fs.azure.account.auth.type":"OAuth",
"fs.azure.account.oauth.provider.type":"org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id":"<client-id>",
"fs.azure.account.oauth2.client.secret":'<secretkey>',
"fs.azure.account.oauth2.client.endpoint":"https://login.microsoftonline.com/<tenant-id>/oauth2/token"}


mount_name = "/mnt/ecomproject2"
mounted_paths = [mount.mountPoint for mount in dbutils.fs.mounts()]

if mount_name not in mounted_paths:
    dbutils.fs.mount(
    source = "abfss://landing-zone-2@ecomadlsbusa.dfs.core.windows.net",     #container@storageacc
    mount_point= mount_name,
    extra_configs=configs
                )

else:
    print(f"{mount_name} is already mounted.")

/mnt/ecomproject2 is already mounted.


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
spark = SparkSession.builder.appName("Ecomdatapipelinechallenge").getOrCreate()

In [ ]:
buyers_df = spark.read.format("parquet")\
                 .option("header","true")\
                 .option("inferSchema","true")\
                 .load("/mnt/ecomproject2/buyers-raw-2/parquet")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1084610385193812>, line 4
      1 buyers_df = spark.read.format("parquet")\
      2                  .option("header","true")\
      3                  .option("inferSchema","true")\
----> 4                  .load("/mnt/ecomproject2/buyers-raw-2/parquet")

File /databricks/spark/python/pyspark/instrumentation_utils.py:47, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     45 start = time.perf_counter()
     46 try:
---> 47     res = func(*args, **kwargs)
     48     logger.log_success(
     49         module_name, class_name, function_name, time.perf_counter() - start, signature
     50     )
     51     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:312, in DataFrameReader.load(self, path, format, schema, **options)
    310 self.options(**options)
    311 if isinstance(path, str):
--> 312     re

In [ ]:
#Saving bronze delta table for buyers

buyers_df.write.format("delta")\
         .mode("overwrite")\
         .option("mergeSchema", "true")\
         .save("/mnt/delta/tables/ecom2/bronze/buyers")

com.databricks.backend.common.rpc.CommandCancelledException
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$5(SequenceExecutionState.scala:136)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3(SequenceExecutionState.scala:136)
	at com.databricks.spark.chauffeur.SequenceExecutionState.$anonfun$cancel$3$adapted(SequenceExecutionState.scala:133)
	at scala.collection.immutable.Range.foreach(Range.scala:158)
	at com.databricks.spark.chauffeur.SequenceExecutionState.cancel(SequenceExecutionState.scala:133)
	at com.databricks.spark.chauffeur.ExecContextState.cancelRunningSequence(ExecContextState.scala:727)
	at com.databricks.spark.chauffeur.ExecContextState.$anonfun$cancel$1(ExecContextState.scala:445)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.spark.chauffeur.ExecContextState.cancel(ExecContextState.scala:445)
	at com.databricks.spark.chauffeur.ChauffeurState.cancelExecutio

In [ ]:
sellers_df = spark.read.format("parquet")\
                  .option("header", "true")\
                  .option("inferSchema", "true")\
                  .load("/mnt/ecomproject2/sellers-raw-2/parquet")

In [ ]:
sellers_df.write.format("delta")\
          .mode("overwrite")\
          .option("mergeSchema", "true")\
          .save("/mnt/delta/tables/ecom2/bronze/sellers")

In [ ]:
country_df = spark.read.format("parquet")\
                  .option("header", "true")\
                  .option("inferSchema", "true")\
                  .load("/mnt/ecomproject2/countries-raw-2/parquet")

In [ ]:
country_df.write.format("delta")\
          .mode("overwrite")\
          .option("mergeSchema", "true")\
          .save("/mnt/delta/tables/ecom2/bronze/country")